In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import config

## Extract CSVs into DataFrame

In [2]:
country_socioeconomic_file = "Resources/country_profile_variables.csv"
country_socioeconomic_df = pd.read_csv(country_socioeconomic_file)
country_socioeconomic_df.head()

,country,Region,Population in thousands (2017),"Population density (per km2, 2017)",GDP: Gross domestic product (million current US$),GDP per capita (current US$)
0,Afghanistan,SouthernAsia,35530,54.4,20270,623.2
1,Albania,SouthernEurope,2930,106.9,11541,3984.2
2,Algeria,NorthernAfrica,41318,17.3,164779,4154.1
3,American Samoa,Polynesia,56,278.2,-99,-99.0
4,Andorra,SouthernEurope,77,163.8,2812,39896.4


In [3]:
country_olympics_file = "Resources/olympics_medals_country_wise.csv"
country_olympics_df = pd.read_csv(country_olympics_file)
country_olympics_df.head()

,country,summer_participations,summer_gold,summer_silver,summer_bronze,summer_total,winter_participations,winter_gold,winter_silver,winter_bronze,winter_total,total_participation,total_gold,total_silver,total_bronze,total_total
0,Afghanistan,15,0,0,2,2,0,0,0,0,0,15,0,0,2,2
1,Algeria,14,5,4,8,17,3,0,0,0,0,17,5,4,8,17
2,Argentina,25,21,26,30,77,20,0,0,0,0,45,21,26,30,77
3,Armenia,7,2,8,8,18,8,0,0,0,0,15,2,8,8,18
4,Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


## Transform country_socioeconomic DataFrame

In [4]:
# Create a filtered dataframe from specific columns
country_socioeconomic_cols = ["country","Population in thousands (2017)","GDP: Gross domestic product (million current US$)"]
country_socioeconomic_transformed = country_socioeconomic_df[country_socioeconomic_cols].copy()

# Rename the column headers
country_socioeconomic_transformed= country_socioeconomic_transformed.rename(columns={
                                                          "Population in thousands (2017)": "population",
                                                          "GDP: Gross domestic product (million current US$)": "gdp"
                                                          })

# Clean the data by dropping duplicates and NA values and setting the index
new_country_socioeconomic_transformed = country_socioeconomic_transformed.dropna(how="any")
new_country_socioeconomic_transformed.drop_duplicates("country", inplace=True)
new_country_socioeconomic_transformed.head()

,country,population,gdp
0,Afghanistan,35530,20270
1,Albania,2930,11541
2,Algeria,41318,164779
3,American Samoa,56,-99
4,Andorra,77,2812


## Transform country_olympics DataFrame

In [5]:
# Create a filtered dataframe from specific columns
country_olympics_cols = ["country", "summer_total", "winter_total", "total_participation", "total_total "]
country_olympics_transformed = country_olympics_df[country_olympics_cols]

# Rename the column headers
country_olympics_transformed= country_olympics_transformed.rename(columns={
                                                          "total_total ": "total_won"
                                                              })

# Clean the data by dropping duplicates and NA values and setting the index
new_country_olympics_transformed = country_olympics_transformed.dropna(how="any")
new_country_olympics_transformed.drop_duplicates("country", inplace=True)
new_country_olympics_transformed.head()

,country,summer_total,winter_total,total_participation,total_won
0,Afghanistan,2,0,15,2
1,Algeria,17,0,17,17
2,Argentina,77,0,45,77
3,Armenia,18,0,15,18
4,Australasia,12,0,2,12


## Connect to local database

In [6]:
protocol = 'postgresql'
username = config.Username
password = config.Password
host = 'localhost'
port = 5432
database_name = 'olympics'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

## Inspect Tables

In [7]:
insp.get_table_names()

['country_socioeconomic', 'country_olympics']

## Load

In [8]:
new_country_socioeconomic_transformed.to_sql(name='country_socioeconomic', con=engine, if_exists='append', index=False)

In [9]:
new_country_olympics_transformed.to_sql(name='country_olympics', con=engine, if_exists='append', index=False)

## Confirm data has been added by querying the country socioeconomic table

In [11]:
pd.read_sql_query('select * from country_socioeconomic', con=engine).head()

,country,population,gdp
0,Afghanistan,35530,20270.0
1,Albania,2930,11541.0
2,Algeria,41318,164779.0
3,American Samoa,56,-99.0
4,Andorra,77,2812.0


## Confirm data has been added by querying the country olympics table

In [12]:
pd.read_sql_query('select * from country_olympics', con=engine).head()

,country,summer_total,winter_total,total_participation,total_won
0,Afghanistan,2,0,15,2
1,Algeria,17,0,17,17
2,Argentina,77,0,45,77
3,Armenia,18,0,15,18
4,Australasia,12,0,2,12


## Join Data from country socioeconomic and country olympics table

In [13]:
sql_join = r"""SELECT cs.country, cs.population, cs.gdp, co.summer_total, co.winter_total, co.total_participation, co.total_won
FROM country_socioeconomic as cs
INNER JOIN country_olympics as co
ON cs.country=co.country;"""
pd.read_sql_query(sql_join, con=engine).head(20)

,country,population,gdp,summer_total,winter_total,total_participation,total_won
0,Afghanistan,35530,20270.0,2,0,15,2
1,Algeria,41318,164779.0,17,0,17,17
2,Argentina,44271,632343.0,77,0,45,77
3,Armenia,2930,10529.0,18,0,15,18
4,Australia,24451,1230859.0,547,19,47,566
5,Austria,8736,376967.0,96,250,52,346
6,Azerbaijan,9828,53049.0,49,0,14,49
7,Bahamas,395,8854.0,16,0,17,16
8,Bahrain,1493,31126.0,4,0,10,4
9,Barbados,286,4385.0,1,0,13,1
